**Stroke Prediction**

Kişinin gelecekte inme yaşayıp yaşamayacağını tahmin etmeye çalışan bir veri bilimi projesidir.

**Neden bu proje?**

İnme dünya genelinde ölüm sebeplerinin başında geliyor.
Erken risk tespiti yapılırsa önleyici tedbirler alınabilir (yaşam tarzı değişiklikleri, ilaç vb.).
Hastaneler, sigorta şirketleri, sağlık kuruluşları bu tahmini kullanabiliyor.

Not: İnme ve felç aynı şey değildir. İnme, beyne giden kan akışının aniden kesilmesi veya azalmasıdır.
Bu durum oksijen eksikliğine yol açar ve beyin hücreleri ölür. Felç, kasların hareket edememesidir.
Felç, inmenin sonucunda ortaya çıkabilir.


In [ ]:
#Kullanacağımız kütüphaneleri ekledik.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings

# Sadece FutureWarning türündeki uyarıları görmezden gel
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#Csv dosyamızı yükledik ve tanımladık.

df = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
#Tüm veriyi yazdırdık.

print(df)

In [ ]:
#Verilerimiz hakkında özet aldık.

df.info()

In [ ]:
#Veri temizleme adımına geçmeden önce nerelerde eksik olduğuna baktık (sütun bazında).

print(df.isnull().sum())

In [ ]:
#Eksik bmi değerinin veri setindeki eksikliği; (yüzde olarak)

df['bmi'].isnull().mean() * 100

In [ ]:
#Eksik veriler %4 olduğu için silindi. (veri setinde çok büyük bir kayıp değil modeli etkilemez)

df = df.dropna(subset=['bmi'])

In [ ]:
#Tekrardan eksik kaldı mı kontrolü

print(df.isnull().sum())

In [ ]:
#Boxplot görselleştirmesi için sayısal değerden farklı sütun var mı baktık.

df.select_dtypes(exclude='number').columns

In [ ]:
#Eksikler tamamlandı.
#Sütunlarda aykırı değer tespiti için önce boxplot görselleştirmesi yaptık.

# Sayısal sütunları seç (id hariç)

numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
if 'id' in numeric_cols:
    numeric_cols.remove('id')

# Grafik yerleşimi için satır ve sütun sayısını hesapla

num_cols = 3  # Yan yana 3 grafik olsun
num_rows = math.ceil(len(numeric_cols) / num_cols)

plt.figure(figsize=(15, 5 * num_rows))

for i, col in enumerate(numeric_cols):
    plt.subplot(num_rows, num_cols, i + 1)
    sns.boxplot(y=df[col], color='skyblue') # Dikey boxplot daha yaygındır çoklu gösterimde
    plt.title(col, fontsize=12)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('all_columns_boxplot.png')

In [ ]:
#Boxplotta görünen glikoz seviyesi için iqr aykırı değer tespiti yaptık.

col_name = 'avg_glucose_level'

#IQR Hesaplama

Q1 = df[col_name].quantile(0.25)
Q3 = df[col_name].quantile(0.75)
IQR = Q3 - Q1

alt_sinir = Q1 - 1.5 * IQR
ust_sinir = Q3 + 1.5 * IQR

#Aykırı ve Normal Verileri Ayırma

aykiri_olanlar = df[(df[col_name] < alt_sinir) | (df[col_name] > ust_sinir)]
normal_olanlar = df[(df[col_name] >= alt_sinir) & (df[col_name] <= ust_sinir)]

print(f"Alt Sınır: {alt_sinir:.2f}")
print(f"Üst Sınır: {ust_sinir:.2f}")
print(f"Tespit Edilen Aykırı Değer Sayısı: {len(aykiri_olanlar)}")

#Görselleştirme (Scatter Plot)

plt.figure(figsize=(12, 6))

#Normal verileri mavi çiz

plt.scatter(normal_olanlar.index, normal_olanlar[col_name], color='blue', label='Normal', alpha=0.5, s=15)

#Aykırı verileri kırmızı çiz

plt.scatter(aykiri_olanlar.index, aykiri_olanlar[col_name], color='red', label='Aykırı Değer', s=30, marker='x')

#Sınır çizgisini ekle

plt.axhline(y=ust_sinir, color='orange', linestyle='--', linewidth=2, label=f'Üst Sınır ({ust_sinir:.1f})')

plt.title('Glikoz Seviyesi - Aykırı Değer Analizi')
plt.xlabel('Kişi Sırası (Index)')
plt.ylabel('Glikoz Seviyesi')
plt.legend()
plt.show()

In [ ]:
#Boxplotta görünen bmi(vücut kitle endeksi) için iqr aykırı değer tespiti yaptık.

col_name = 'bmi'

#IQR Hesaplama

Q1 = df[col_name].quantile(0.25)
Q3 = df[col_name].quantile(0.75)
IQR = Q3 - Q1

alt_sinir = Q1 - 1.5 * IQR
ust_sinir = Q3 + 1.5 * IQR

#Aykırı ve Normal Verileri Ayırma

aykiri_olanlar = df[(df[col_name] < alt_sinir) | (df[col_name] > ust_sinir)]
normal_olanlar = df[(df[col_name] >= alt_sinir) & (df[col_name] <= ust_sinir)]

print(f"Alt Sınır: {alt_sinir:.2f}")
print(f"Üst Sınır: {ust_sinir:.2f}")
print(f"Tespit Edilen Aykırı Değer Sayısı: {len(aykiri_olanlar)}")

#Görselleştirme (Scatter Plot)

plt.figure(figsize=(12, 6))

#Normal verileri mavi çiz

plt.scatter(normal_olanlar.index, normal_olanlar[col_name], color='blue', label='Normal', alpha=0.5, s=15)

#Aykırı verileri kırmızı çiz

plt.scatter(aykiri_olanlar.index, aykiri_olanlar[col_name], color='red', label='Aykırı Değer', s=30, marker='x')

#Sınır çizgisini ekle

plt.axhline(y=ust_sinir, color='orange', linestyle='--', linewidth=2, label=f'Üst Sınır ({ust_sinir:.1f})')

plt.title('Vücut Kitle Endeksi - Aykırı Değer Analizi')
plt.xlabel('Kişi Sırası (Index)')
plt.ylabel('Vücut Kitle Endeksi')
plt.legend()
plt.show()

In [ ]:
#Buradaki aykırı değerler inme inme olasılığını hesaplarken asıl sıkıntı teşkil eden grup olabileceğinden silmedik.

In [ ]:
#Yanlış tipte değer var mı kontrolü

df.dtypes

In [ ]:
#Duplicate (tekrar eden satır) temizleme

df.duplicated().sum()

In [ ]:
#Veri setindeki kategorik değişkenlerdeki farklı olanları inceledik.

for col in df.select_dtypes(include=['object']).columns:
    print(col, ":", df[col].unique())

In [ ]:
df['gender'].value_counts()

In [ ]:
#1 tane other değerinin modele katkısı olamadığı için siliyoruz

df = df[df['gender'] != 'Other']

In [ ]:
#Kişinin sigara içip içmediğini bilmemesi doğru bir parametre olmadığı için NAN değere çevrilir.

df['smoking_status'] = df['smoking_status'].replace('Unknown', np.nan)

In [ ]:
#Düzenlediğimiz veri setini inceleme adımına geçiyoruz.

In [ ]:
#Projemiz bir sınıflandırma problemi olduğundan genel inme dağılımını incelemekle başlıyoruz

stroke_count = df["stroke"].sum()
no_stroke_count = len(df) - stroke_count

labels = ["İnme geçiren", "İnme geçirmeyen"]
sizes = [stroke_count, no_stroke_count]

plt.figure(figsize=(4,4))
plt.pie(sizes, labels=labels, autopct="%1.1f%%", startangle=90)
plt.title("İnme Durumunun Yüzdelik Dağılımı")
plt.tight_layout()
plt.show()
#Veri setindeki kişilerin yalnızca %3.2’i felç geçirmiş. Bu nedenle dengesiz bir veri söz konusu.

In [ ]:
#Kategorik değişkenlerin genel dağılımını ve bunların inme ile ilişkilerini incelemeye başlayalım

kategorik_degiskenler = ['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status']

for col in kategorik_degiskenler:
    print("*****", col, "*****")
    print(df[col].value_counts())
    print("--- İnme oranı (%) ---")
    print(df.groupby(col)['stroke'].mean().sort_values(ascending=False) * 100)
    print()

In [ ]:
# Veri setindeki kategorik değişkenlerin inme ile ilişkisi incelendiğinde:

#Hipertansiyon ve kalp hastalığı inme riskini en güçlü şekilde artıran iki değişkendir. Her iki durumda da inme oranı normallerin yaklaşık 4 katıdır.
#Evli olmak, serbest çalışan olmak ve önceden sigara kullanan grubunda yer almak inme oranlarını artırmaktadır; ancak bu artışların büyük kısmının yaş etkisi ile ilişkili olduğu düşünülmektedir.
#Cinsiyet ve yaşanılan yer (kırsal/şehir) inme riski açısından belirgin bir farklılık göstermemektedir.

In [ ]:
#Önemli gördüğümüz kategorilerin inme ile ilişkilerini görsel olarakta görelim
#### Hipertansiyon ve inme oranı
stroke_rate = df.groupby("hypertension")["stroke"].mean()
stroke_rate.plot(kind='bar', color=['lightgreen','pink'])
plt.xlabel("Hipertansiyon")
plt.ylabel("İnme Oranı")
plt.title("Hipertansiyon Durumuna Göre İnme Oranı")
plt.xticks([0,1], ["Hipertansiyon Yok", "Hipertansiyon Var"], rotation=0)
plt.show()
#Grafikten anlayabileceğimiz üzere hipertansiyonu olan kişilerin inme geçirme oranının daha yüksek olduğunu gözlemliyoruz

In [ ]:
#### Kalp rahatsızlığı ve inme oranı
stroke_rate = df.groupby("heart_disease")["stroke"].mean()
stroke_rate.plot(kind='bar', color=['lightgreen','pink'])
plt.xlabel("Kalp Hastalığı")
plt.ylabel("İnme Oranı")
plt.title("Kalp Hastalığı Durumuna Göre İnme Oranı")
plt.xticks([0,1], ["Kalp Hastalığı Yok", "Kalp Hastalığı Var"], rotation=0)
plt.show()

In [ ]:
#Sayısal değişkenlerin temel analizi
sayisal_degiskenler = ['age', 'bmi', 'avg_glucose_level']

for col in sayisal_degiskenler:
    print("*****", col, "*****")
    print(df[col].describe())
    print("--- İnme durumuna göre ortalama ---")
    print(df.groupby("stroke")[col].mean())

In [ ]:
# Yaş, inme için en güçlü sayısal göstergedir.
# İnme geçirenlerin ortalama yaşı 66, geçirmeyenlerin 40 civarındadır.
# Bu belirgin fark yaşlandıkça inme riskinin arttığını gösterir.

# BMI, inme geçirenlerde biraz daha yüksektir ancak fark çok büyük değildir.
# İlişki vardır ama güçlü değildir.

# Glikoz seviyesi ile inme arasında belirgin bir fark yoktur.
# Bu değişken zayıf bir ilişki göstermektedir.

In [ ]:
# İnme riskinin hangi yaş aralıklarında olduğunu görebilmek adına seviyelere ayıralım ve grafiğini oluşturalım
age_bins = [0, 17, 35, 50, 65, 100]
age_labels = ['Çocuk (0-17)', 'Genç (18-35)', 'Orta Yaş (36-50)', 'Yaşlı (51-65)', 'İleri Yaş (66+)']

df['age_category'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=True)
inme_orani_age = df.groupby('age_category', observed=False)['stroke'].mean() * 100

In [ ]:
#Grafiktende anlayabileceğimiz şekilde ileri yaş aralığında inme oranı çok yüksek 
plt.figure(figsize=(6,4))

ax = sns.barplot(
    x=inme_orani_age.index, 
    y=inme_orani_age.values, 
    palette="Reds", 
    hue=inme_orani_age.index, 
    dodge=False
)

# Legend'ı (göstergeyi) kaldırmak için güvenli yöntem:
# Eğer hue kullandığınız için bir legend oluştuysa ve silmek istiyorsanız:
if ax.legend_:
    ax.legend_.remove()

plt.ylabel("İnme Oranı (%)")
plt.xlabel("Yaş Kategorisi")
plt.show()

In [ ]:
#Sayısal değişkenlerin arasındaki korelasyonu görmek adına ısı grafiği oluşturduk
sns.heatmap(df[['age','bmi','avg_glucose_level','hypertension','heart_disease','stroke']].corr(), annot=True)
# Grafikten elde ettiklerimiz 
# 1- Yaş arttıkça vücut kitle endeksi artma eğiliminde
# 2- Yaş arttıkça inme olasılığı artıyor
# 3- glukoz leveli ile inme arasında doğrudan bir ilişki yok
# 4- vücut kitle endeksi ile inme arasında çok anlamlı bir korelasyon yok
# 5- yaş arttıkça hipertansiyon hastalığı artıyor
# 6- yaş arttıkça kalp hastalığıda artıyor

In [ ]:
#Şimdi inmeyi etkileyen değişkenlerin birbiri ile ilişkisine bakıp inme oranını nasıl etkilediğini inceleyeceğiz

In [ ]:
#Yaş aralıklarına göre hipertansiyonu olan kişilerin ve olmayan kişilerin inme geçirme oranı
pivot = df.groupby(['age_category','hypertension'],observed=False)['stroke'].mean().unstack() * 100
pivot.plot(kind='bar', figsize=(6,4), colormap='viridis')
plt.ylabel("İnme Oranı (%)")
plt.xlabel("Yaş Kategorisi")
plt.title("Yaş Kategorisi ve Hipertansiyona Göre İnme Oranı")
plt.xticks(rotation=45)
plt.legend(["Hipertansiyon Yok (0)", "Hipertansiyon Var (1)"])
plt.show()
# Yaşın artması ve hipertansiyon hastalığının varlığı inme riskini arttırıyor

In [ ]:
pivot = df.groupby(['age_category','heart_disease'],observed=False)['stroke'].mean().unstack() * 100
pivot.plot(kind='bar', figsize=(6,4), colormap='winter_r')
plt.ylabel("İnme Oranı (%)")
plt.xlabel("Yaş Kategorisi")
plt.title("Yaş Kategorisi ve Kalp Hastalığına Göre İnme Oranı")
plt.xticks(rotation=45)
plt.legend(["Kalp Hastalığı Yok (0)", "Kalp Hastalığı Var (1)"])
plt.show()
# Yaş ilerledikçe kalp hastalığı artıyor 51-65 yaş arası kalp hastalığı olanların olmayanlara göre inme geçirme oranı tam 4 kat fazla

In [ ]:
pivot = df.groupby(['smoking_status','hypertension'])['stroke'].mean().unstack() * 100
pivot.plot(kind='bar', figsize=(6,4), colormap='magma')
plt.ylabel("İnme Oranı (%)")
plt.xlabel("Sigara İçme Durumu")
plt.title("Sigara İçme Durumu ve Hipertansiyona Göre İnme Oranı")
plt.xticks(rotation=45)
plt.legend(["Hipertansiyon Yok (0)", "Hipertansiyon Var (1)"])
plt.show()
# 

In [ ]:
# evli olma durumunun yaş ile inme oranına bir etkisi varmı gözlemleyelim
pivot = df.groupby(['ever_married','age_category'],observed=False)['stroke'].mean().unstack() * 100
pivot.plot(kind='bar', figsize=(6,4), colormap='magma')
plt.title("Evli Olma Durumu ve Yaş Kategorisine Göre İnme Oranı (%)")
plt.ylabel("İnme Oranı (%)")
plt.xlabel("Evli Olma Durumu")
plt.xticks(rotation=0)
plt.legend(title="Yaş Kategorisi")
plt.tight_layout()
plt.show()
# İnme oranları hem evli hem evli olmayan bireylerde yaşla birlikte artmaktadır.
# Bu artış büyük ölçüde yaşın doğal etkisinden kaynaklanmaktadır.
# Evli olmayan grubun özellikle ileri yaş kategorisindeki örneklem sayısının düşük olması,
# inme oranlarının sağlıklı şekilde karşılaştırılmasını zorlaştırmaktadır.
# Bu nedenle evlilik durumu ile inme arasında doğrudan bir ilişki var diyemeyiz.

In [ ]:
pivot = df.groupby(['work_type','age_category'],observed=False)['stroke'].mean().unstack() * 100

pivot.plot(kind='bar', figsize=(6,4), colormap='magma')
plt.title("Çalışma Şekli ve Yaş Kategorisine Göre İnme Oranı (%)")
plt.ylabel("İnme Oranı (%)")
plt.xlabel("Çalışma Şekli")
plt.xticks(rotation=45)
plt.legend(title="Yaş Kategorisi")
plt.tight_layout()
plt.show()
# Çalışma tipi kategorileri incelendiğinde tüm gruplarda inme oranlarının yaşla birlikte arttığı görülmektedir. Bu durum, yaşın çalışılan sektörden bağımsız olarak stroke için temel belirleyici faktör 
# olduğunu göstermektedir.Dolayısıyla çalışma tipi tek başına inme riskini belirleyen bir faktör olarak görünmemektedir

In [ ]:
# # Genel Analiz Sonuçları Neler Yapıldı Ne Elde Edildi

# 1- Veri Setinin İncelenmesi
# Veri seti yüklenmiş ve ilk gözlemler incelenmiştir.
# Kategorik ve sayısal değişkenlerin dağılımları kontrol edilmiştir.
# Eksik veri bulunmadığı görülmüştür.

# 2. Genel İnme Dağılımı
# Veri setindeki genel inme oranı hesaplanmış ve dengesiz bir dağılım olduğu gözlemlenmiştir.

# 3. Kategorik Değişken Analizleri
# Aşağıdaki kategorik değişkenlerin inme ile ilişkileri incelenmiştir:
# Cinsiyet
# Evli olup olmama durumu
# Çalışma şekli
# Konum (kırsal/şehir)
# Sigara kullanımı

# 3.1. Kategorik Değişkenlerin Analizinden Çıkarımlar
# Hipertansiyon ve kalp hastalığı, inme riskini en fazla artıran faktörlerdir.
# Evli olmak, serbest çalışma, geçmişte sigara içmek gibi durumlar riskle ilişkili görünse de büyük oranda yaş etkisinden kaynaklanmaktadır.
# Cinsiyet ve yaşanılan yer inme üzerinde belirgin farklılık yaratmamaktadır.

# 5. Sayısal Değişken Analizleri
# Özellikle yaş, inme riskinin belirgin şekilde arttığı temel değişkendir.
# Yaş arttıkça hipertansiyon ve kalp hastalığının görülme sıklığı da arttığından, bu değişkenler arasında ilişkiler gözlenmiştir.

# 6. Genel Sonuç
# Bu proje sonucunda, inme riskini en çok etkileyen temel faktörler belirlenmiştir:
# Hipertansiyon
# Kalp hastalığı
# Yaş
# Diğer değişkenlerin etkileri daha sınırlı görünmekte ve çoğunlukla yaş ile ilişkili ikincil yansımalar olarak değerlendirilmektedir.

In [ ]:
!pip uninstall scikit-learn imbalanced-learn -y
!pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1

In [ ]:
#EDA kısmı tamamlandı ML adımına geçiyoruz.

#Veri setimiz dengesiz olduğu için doğruluk (accuracy) yerine duyarlılık (Recall) ve F1-Score gibi metriklere odaklanacağız.
#Adım 1: Veri Ön İşleme (Preprocessing)

#Modellerin sayısal verilerle çalışabilmesi için kategorik verileri dönüştürmemiz ve gereksiz sütunları atmamız gerekir.

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. Gereksiz sütunları çıkaralım
# 'id' tahmin için gereksizdir. 
# 'age_category' görselleştirme içindi, model için orjinal sayısal 'age' sütunu daha değerlidir.
df_ml = df.drop(['id', 'age_category'], axis=1, errors='ignore')

# 2. Kategorik Değişkenleri Dönüştürme (One-Hot Encoding)
# Gender, marriage, work_type gibi text verileri sayıya çeviriyoruz.
df_ml = pd.get_dummies(df_ml, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], drop_first=True)

# 3. Bağımlı (y) ve Bağımsız (X) Değişkenleri Ayırma
X = df_ml.drop('stroke', axis=1) # Özellikler
y = df_ml['stroke']              # Hedef (Tahmin edilecek)

# 4. Eğitim ve Test Setine Ayırma
# Verinin %80'i ile modeli eğiteceğiz, %20'si ile test edeceğiz.
# stratify=y kullanıyoruz ki inme geçirenlerin oranı her iki sette de dengeli olsun.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 5. Özellik Ölçeklendirme (Scaling)
# Yaş 80, glikoz 200 olabilir ama hipertansiyon 0-1 arasındadır. 
# Bu farkı kapatmak için standartlaştırma yapıyoruz.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Eğer ölçeklendirme yapmasaydık; model, 220 sayısını 1 sayısından matematiksel olarak çok büyük göreceği için, "Şeker seviyesi, tansiyondan 220 kat daha önemli" gibi yanlış bir karar verecekti. Bu yüzden eşit şarta getirdik.

In [ ]:
#Adım 2: Dengesiz Veri ile Başa 

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# 2. Model Kurma
model = RandomForestClassifier(class_weight='balanced', random_state=42)
#class_weight balanced kullanarak yanlışta cezayı artırma gibi bi parametre oldu yani

# Modeli eğit
model.fit(X_train, y_train)

# Tahmin ve Rapor
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

#Matrisi hesapla
cm = confusion_matrix(y_test, y_pred)

#Görselleştir
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Tahmin Edilen (0: Sağlıklı, 1: İnme)')
plt.ylabel('Gerçek Durum (0: Sağlıklı, 1: İnme)')
plt.title('Modelin Hata Matrisi')
plt.show()

In [ ]:
#Özellik önem düzeylerini al

feature_importances = pd.Series(model.feature_importances_, index=X.columns)

#En önemli 10 özelliği görselleştir
plt.figure(figsize=(10, 6))
feature_importances.nlargest(10).plot(kind='barh', color='teal')
plt.title('İnme Riskini Belirleyen En Önemli Faktörler')
plt.xlabel('Önem Derecesi')
plt.show()

In [ ]:
#RandomForest modelini kurduk şimdi birde lojistik regresyon modeli kuracağız ve hangi modelin recall değeri daha yüksek buna bakacağız.

In [ ]:
from sklearn.linear_model import LogisticRegression

# 1. Modeli Kur
log_model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

# 2. Eğit
log_model.fit(X_train, y_train)

# 3. Tahmin Et
y_pred_log = log_model.predict(X_test)

# 4. Sonuçları Kıyasla
print("--- Logistic Regression Sonuçları ---")
print(classification_report(y_test, y_pred_log))

In [ ]:
#Modelleri karşılaştırıyoruz.

from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, roc_curve

# 1. Metrikleri Hesaplama Fonksiyonu
def get_metrics(model_name, y_true, y_pred, y_prob):
    return {
        'Model': model_name,
        'Accuracy (Doğruluk)': accuracy_score(y_true, y_pred),
        'Recall (Duyarlılık)': recall_score(y_true, y_pred),
        'F1-Score': f1_score(y_true, y_pred),
        'AUC Score': roc_auc_score(y_true, y_prob)
    }

#Random Forest Tahminleri
y_pred_rf = model.predict(X_test)
y_prob_rf = model.predict_proba(X_test)[:, 1] # Olasılık değerleri (1 sınıfı için)

#Lojistik Regresyon Tahminleri
y_pred_log = log_model.predict(X_test)
y_prob_log = log_model.predict_proba(X_test)[:, 1]

#Sonuçları Tabloya Dökme
results = pd.DataFrame([
    get_metrics('Random Forest', y_test, y_pred_rf, y_prob_rf),
    get_metrics('Logistic Regression', y_test, y_pred_log, y_prob_log)
])

print("--- Model Karşılaştırma Tablosu ---")
print(results.round(3))

# --- 2. ROC Eğrisi Çizimi (Görsel Karşılaştırma) ---
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
fpr_log, tpr_log, _ = roc_curve(y_test, y_prob_log)

plt.figure(figsize=(8, 6))

# Random Forest Çizgisi
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_score(y_test, y_prob_rf):.2f})', color='green')

# Logistic Regression Çizgisi
plt.plot(fpr_log, tpr_log, label=f'Logistic Regression (AUC = {roc_auc_score(y_test, y_prob_log):.2f})', color='blue')

# Rastgele Tahmin Çizgisi (Referans)
plt.plot([0, 1], [0, 1], 'k--', label='Rastgele Tahmin')

plt.xlabel('False Positive Rate (Yanlış Alarm Oranı)')
plt.ylabel('True Positive Rate (Gerçek Yakalama Oranı)')
plt.title('ROC Eğrisi: Hangi Model Daha İyi?')
plt.legend()
plt.show()

In [ ]:
#Lojistik regresyon daha iyi olduğu için bu model üstünden confusion matrix çıkaracağız.

from sklearn.metrics import precision_recall_curve

def plot_precision_recall_vs_threshold(y_test, y_probs, model_name):
    # Precision, Recall ve Threshold değerlerini hesapla
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)
    
    # Grafiği çiz
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision (Kesinlik)", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall (Duyarlılık - Hasta Yakalama)", linewidth=2)
    
    plt.xlabel("Eşik Değeri (Threshold)", fontsize=12)
    plt.ylabel("Skor", fontsize=12)
    plt.title(f"{model_name} İçin En İyi Eşik Değerini Bulma", fontsize=14)
    plt.legend(loc="center left", fontsize=12)
    plt.grid(True)
    plt.show()

#Lojistik Regresyon için Analiz
#y_prob_log değişkeni önceki adımlardan hafızada olmalı
plot_precision_recall_vs_threshold(y_test, y_prob_log, "Logistic Regression")

In [ ]:
#Final Modeli

final_threshold = 0.3

#Modelin ürettiği olasılıkları bu eşiğe göre 1 veya 0'a çeviriyoruz
y_pred_final = (y_prob_log >= final_threshold).astype(int)

#Sonuçları Raporlama
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print(f"--- SEÇİLEN EŞİK DEĞERİ: {final_threshold} ---")
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred_final))

#Final Confusion Matrix
plt.figure(figsize=(6, 5))
cm_final = confusion_matrix(y_test, y_pred_final)
sns.heatmap(cm_final, annot=True, fmt='d', cmap='Greens', cbar=False)
plt.title(f'Final Model Performansı (Eşik: {final_threshold})')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek Durum')
plt.show()